In [1]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# Asegúrate de que el modelo esté entrenado y cargado correctamente
modelWin_OCR = load_model('modelWin.h5')  # Carga el modelo que tienes en memoria (modelWin)

# Dimensiones esperadas por el modelo
img_height = 32
img_width = 32

# Inicializar cámara
cap = cv2.VideoCapture(0)  # Usa 1 o 2 si tu webcam no está en el índice 0

if not cap.isOpened():
    raise IOError("❌ No se pudo acceder a la cámara. Verifica que esté conectada y no usada por otra app.")

print("Cámara iniciada. Presiona 't' para tomar una foto y 'q' para salir.")

while True:
    # Leer un frame de la cámara
    ret, frame = cap.read()
    if not ret:
        print("⚠️ No se pudo capturar el frame.")
        break

    # Mostrar la vista en vivo de la cámara
    cv2.imshow("Vista en Vivo", frame)

    # Esperar por una tecla para capturar la imagen
    key = cv2.waitKey(1) & 0xFF

    # Tomar foto cuando se presione la tecla 't'
    if key == ord('t'):
        start_time = time.time()  # Iniciar cronómetro para medir el tiempo de procesamiento
        
        # 1. Convertir a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # 2. Desenfoque para reducir el ruido
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)

        # 3. Umbral fijo o un umbral adaptativo con parámetros ajustados
        _, thresh = cv2.threshold(blurred, 120, 255, cv2.THRESH_BINARY_INV)

        # 4. Reducción de la dilatación para evitar deformación
        kernel = np.ones((2, 2), np.uint8)
        dilated = cv2.dilate(thresh, kernel, iterations=1)

        # 5. Buscar contornos
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        digits = []  # Para almacenar los dígitos detectados
        result_frame = frame.copy()  # Copiar la imagen para mostrarla después

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            if w < 10 or h < 10:
                continue  # Ignora ruido pequeño

            roi = dilated[y:y+h, x:x+w]
            roi_resized = cv2.resize(roi, (img_width, img_height))
            roi_resized = roi_resized.astype("float32") / 255.0
            roi_resized = np.expand_dims(roi_resized, axis=-1)  # Canal
            roi_resized = np.expand_dims(roi_resized, axis=0)   # Batch

            # Verificación de forma de la entrada antes de la predicción
            print(f"Forma de la entrada al modelo: {roi_resized.shape}")

            # Predicción con el modelo
            pred = modelWin_OCR.predict(roi_resized, verbose=0)
            label = np.argmax(pred)
            conf = np.max(pred)

            # Verificar si la confianza es suficientemente alta
            if conf > 0.95:
                # Dibujar predicción
                cv2.rectangle(result_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                text = f'{label} ({conf:.2f})'
                cv2.putText(result_frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.7, (0, 0, 255), 2)

                # Guardar el dígito detectado en la lista
                digits.append(label)

        # Mostrar la imagen con los resultados
        cv2.imshow("Resultado OCR", result_frame)
        
        # Mostrar estadísticas
        processing_time = time.time() - start_time
        print(f"Procesado en {processing_time:.2f}s | Dígitos: {len(digits)}")
        
        # Opción para guardar
        save = input("¿Guardar resultado? (s/n): ").lower()
        if save == 's':
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = f"ocr_result_{timestamp}.jpg"
            cv2.imwrite(filename, result_frame)
            print(f"Imagen guardada como {filename}")

    # Salir cuando se presiona 'q'
    if key == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()

Cámara iniciada. Presiona 't' para tomar una foto y 'q' para salir.
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Procesado en 1.67s | Dígitos: 2
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Forma de la entrada al modelo: (1, 32, 32, 1)
Procesado en 1.21s | Dígitos: 4
Imagen guardada como ocr_result_20250510